In [24]:
# Import Statements

import pandas as pd 
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import os
import os.path as path
from os.path import dirname as up
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf


pd.set_option('display.max_columns', None)

In [77]:
# Absolute Paths
master_folder = up(up(os.path.abspath(os.getcwd()))) + "\\"
production_pipeline = "production_pipeline\\"
data_folder = "data\\"
code_folder = "code\\"
raw_data_folder = "raw_data\\"
processed_input_data = "processed_input_data\\"
temp_folder = "temp_input_data\\"
store_prd = "StoreProduct.csv"
raw_data = "ExpressTop50.csv"


In [42]:
def create_folder_structure():
    # Checking if production pipeline folder present.
    if not os.path.isdir(master_folder+production_pipeline):
        os.makedirs(master_folder+production_pipeline)
        print("CREATED: " + master_folder+production_pipeline)
    else:
        print(master_folder+production_pipeline + " is present")

    # Checking if data folder present.
    if not os.path.isdir(master_folder+production_pipeline+data_folder):
        os.makedirs(master_folder+production_pipeline+data_folder)
        print("CREATED: " + master_folder+production_pipeline+data_folder)
    else:
        print(master_folder+production_pipeline+data_folder + " is present")

    # Checking if code folder present.
    if not os.path.isdir(master_folder+production_pipeline+code_folder):
        os.makedirs(master_folder+production_pipeline+code_folder)
        print("CREATED: " + master_folder+production_pipeline+code_folder)
    else:
        print(master_folder+production_pipeline+code_folder + " is present")

    # Checking if raw data folder present.
    if not os.path.isdir(master_folder+production_pipeline+data_folder+raw_data_folder):
        os.makedirs(master_folder+production_pipeline+data_folder+raw_data_folder)
        print("CREATED: " + master_folder+production_pipeline+data_folder+raw_data_folder)
    else:
        print(master_folder+production_pipeline+data_folder+raw_data_folder + " is present")


    # Checking if processed input data folder present.
    if not os.path.isdir(master_folder+production_pipeline+data_folder+processed_input_data):
        os.makedirs(master_folder+production_pipeline+data_folder+processed_input_data)
        print("CREATED: " + master_folder+production_pipeline+data_folder+processed_input_data)
    else:
        print(master_folder+production_pipeline+data_folder+processed_input_data + " is present")

    # Checking if temp input data folder present.
    if not os.path.isdir(master_folder+production_pipeline+data_folder+temp_folder):
        os.makedirs(master_folder+production_pipeline+data_folder+temp_folder)
        print("CREATED: " + master_folder+production_pipeline+data_folder+temp_folder)
    else:
        print(master_folder+production_pipeline+data_folder+temp_folder + " is present")


In [43]:
create_folder_structure()

c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\ is present
c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\ is present
c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\code\ is present
c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\raw_data\ is present
c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\processed_input_data\ is present
CREATED: c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\temp_input_data\


In [27]:
# Function to convert dataframe in =to downloadable csv file

import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [30]:
prd = 538252
store = 158 

In [ ]:
# Reading the data
str_prd = pd.read_csv(master_folder+production_pipeline+data_folder+raw_data_folder+raw_data+)

# Setting data types

str_prd['p_id'] = str_prd['p_id'].astype(str)
str_prd['p_id'] = str_prd['p_id'].str.strip()

str_prd['ut_id'] = str_prd['ut_id'].astype(str)
str_prd['ut_id'] = str_prd['ut_id'].str.strip()




In [ ]:
# Reading the data for the specific store and product
def read_data_store_prd(prd_id,store_id):
    data = pd.read_csv(master_folder+production_pipeline+data_folder+raw_data_folder + raw_data,chunksize=1000000)
    l = []
    for i in data:

        i = i.loc[(i["p_id"]==prd_id) & (i["ut_id"]==store_id)]
        l.append(i)
    data = pd.concat(l, ignore_index=True)
    return(data)


In [55]:
def get_one_file_data(prd_id,store_id):
    # prd_id = str(p)
    # store_id = str(s)
    
    data = pd.read_csv(master_folder+production_pipeline+data_folder+raw_data_folder + raw_data,chunksize=1000000)
    l = []
    for i in data:
        i = i.loc[(i["p_id"]==prd_id) & (i["ut_id"]==store_id)]
        l.append(i)
    
    data = pd.concat(l, ignore_index=True)
    data["p_id"] = data["p_id"].astype(str)
    data["ut_id"] = data["ut_id"].astype(str)
    
    #Cleaning and pre-processing data

    data.rename(
        {
            'ut_id': 'store_id', 
            'p_id': 'prd_id',
            'mkt_bskt_ut_qt':'quantity'
        }, axis=1, inplace=True)
    
    data = data.drop_duplicates(subset = ["day_dt","store_id","prd_id"])

    if not os.path.exists(master_folder+production_pipeline+data_folder+temp_folder):
        os.makedirs(master_folder+production_pipeline+data_folder+temp_folder)

    
    print(master_folder+production_pipeline+data_folder+temp_folder+"store_"+str(store_id)+"_prd_"+str(prd_id)+".csv")

    return (data)

In [54]:
prd = 538252
store = 158 
x = get_one_file_data(prd_id,store_id)
display(x)

c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\temp_input_data\store_158_prd_538252.csv


,day_dt,wk_end_dt,store_id,prd_id,mjr_mds_are_id,mjr_p_cls_id,p_sell_dur,quantity,p_bse_pr_am,p_promo_pr_am,atl_p_sold_am,ntl_log_mkt_bskt_ut_qt,ntl_log_bse_pr_am,ntl_log_bse_dct_pr_am,ntl_log_bse_promo_dct_pr_am,ut_same_sto_flg,atl_min_temp_val,atl_mean_temp_val,atl_max_temp_val,atl_t_ppt_qt,atl_t_snow_qt,day_bef_ppt_temp_val,day_bef_snow_temp_val,day_aft_ppt_temp_val,day_aft_snow_temp_val,sunprexmas,redsat,blackfri,turkey,saturdayb4turkey,fridayb4turkey,veterans,halloweenfri,halloween,halloweeneve,prehalo_sun,prehalo_fri,columbus,laborfri,laborsun,laborsat,laborwkend,labor,dad,dadsat,memwkend,mom,momsat,easterwk,easter,eastersat,presday,valentinewknd,valentine,dayb4valentine,superbowlsat,superbowl,mlk,ny,julyfour,halloweenwkend,dectwothree,xmaseve,satpny,sunpny,memorial,prehalo_sat,turkeywed,dectwosix,mcc_blk_out,nyeve,fcl_per_of_yr_id,fcl_wk_of_yr_id,fcl_yr_id,cldr_day_of_wk_id,augdow1,augdow2,augdow3,augdow4,augdow5,augdow6,augdow7,julydow1,julydow2,julydow3,julydow4,julydow5,julydow6,julydow7,dow1,dow2,dow3,dow4,dow5,dow6,dow7,wk1,wk2,wk3,wk4,wk5,wk6,wk7,wk8,wk9,wk10,wk11,wk12,wk13,wk14,wk15,wk16,wk17,wk18,wk19,wk20,wk21,wk22,wk23,wk24,wk25,wk26,wk27,wk28,wk29,wk30,wk31,wk32,wk33,wk34,wk35,wk36,wk37,wk38,wk39,wk40,wk41,wk42,wk43,wk44,wk45,wk46,wk47,wk48,wk49,wk50,wk51,wk52,HomeLocationCapacityQuantity
0,2023-01-07,2023-01-07,158,538252,L6-000003,L2-014980,1916,32,1.89,1.89,1.89,3.496508,0.636577,0.0,0.0,Y,28.9,30.5,32.0,0.00,0.0,0.09,-999.99,0.14,0.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,49,2022,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2023-01-06,2023-01-07,158,538252,L6-000003,L2-014980,1916,47,1.89,1.89,1.89,3.871201,0.636577,0.0,0.0,Y,31.0,33.0,35.0,0.14,0.6,0.00,0.00,0.09,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,49,2022,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2023-01-05,2023-01-07,158,538252,L6-000003,L2-014980,1915,70,1.89,1.89,1.89,4.262680,0.636577,0.0,0.0,Y,33.0,34.5,36.0,0.09,0.0,0.14,0.60,0.09,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,49,2022,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2023-01-04,2023-01-07,158,538252,L6-000003,L2-014980,1914,42,1.89,1.89,1.89,3.761200,0.636577,0.0,0.0,Y,33.9,38.5,43.0,0.09,0.0,0.09,0.00,0.33,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,49,2022,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2023-01-03,2023-01-07,158,538252,L6-000003,L2-014980,1913,53,1.89,1.89,1.89,3.988984,0.636577,0.0,0.0,Y,36.0,38.0,40.0,0.33,0.0,0.09,0.00,0.00,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,49,2022,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,2018-09-27,2018-09-29,158,538252,L6-000003,L2-014980,4

In [88]:
def get_all_file_data(start_date):
    str_prd = pd.read_csv(master_folder+production_pipeline+data_folder+raw_data_folder+store_prd)
    str_prd = str_prd[["p_id","ut_id"]]
    str_prd = str_prd.drop_duplicates()
    data = pd.read_csv(master_folder+production_pipeline+data_folder+raw_data_folder + raw_data,chunksize=1000000)
    

    
    for index, row in str_prd.iterrows():
        
        l = []
        for i in data:
            if index == 0:
                i['day_dt'] = pd.to_datetime(i['day_dt'])
            i = i.loc[i["day_dt"]>start_date]
            i = i.loc[(i["p_id"]==row['p_id']) & (i["ut_id"]==row['ut_id'])]
            l.append(i)
        
        data = pd.concat(l, ignore_index=True)
        
        # Cleaning and pre-processing data

        data.rename(
            {
                'ut_id': 'store_id', 
                'p_id': 'prd_id',
                'mkt_bskt_ut_qt':'quantity'
            }, axis=1, inplace=True)
        
        data = data.drop_duplicates(subset = ["day_dt","store_id","prd_id"])

        if not os.path.exists(master_folder+production_pipeline+data_folder+processed_input_data):
            os.makedirs(master_folder+production_pipeline+data_folder+processed_input_data)

        
        print(master_folder+production_pipeline+data_folder+processed_input_data+"store_"+str(row['ut_id'])+"_prd_"+str(row['p_id'])+".csv")
        data.to_csv(master_folder+production_pipeline+data_folder+processed_input_data+"store_"+str(row['ut_id'])+"_prd_"+str(row['p_id'])+".csv",index = False)
        

In [89]:
get_all_file_data("2020-07-12")

c:\Personal Lavanya\Purdue\MSBAIM\IP\production_pipeline\data\processed_input_data\store_19_prd_536565.csv


AttributeError: 'str' object has no attribute 'loc'